# Mongo tutorial

## Prerequisites

### Server status

Check the status of your Mongo cluster : `!sudo service mongod status`

If the cluster is inactive, `!sudo service mongod start` to start it.

In [1]:
!sudo service mongod status

● mongod.service - MongoDB Database Server
   Loaded: loaded (/lib/systemd/system/mongod.service; disabled)
   Active: active (running) since Sun 2019-02-10 11:26:35 GMT; 3h 53min ago
     Docs: https://docs.mongodb.org/manual
 Main PID: 1620 (mongod)
   CGroup: /system.slice/mongod.service
           └─1620 /usr/bin/mongod --config /etc/mongod.conf

Feb 10 11:26:35 nosql systemd[1]: Started MongoDB Database Server.


### Documentation

You will find all documentation for :
* [Mongo commands](https://docs.mongodb.com/manual/reference/)
* [Mongo python client](http://api.mongodb.com/python/current/api/pymongo/mongo_client.html#pymongo.mongo_client.MongoClient)

### Import libraries

In [2]:
import datetime
from pprint import pprint

import pymongo
from pymongo import MongoClient

In [3]:
client = MongoClient('localhost', 27017)

In [4]:
# let's work in a test_database
db = client.test_database
posts = db.posts

In [5]:
post = {
    "author": "Mike",
    "text": "My first blog post!",
    "tags": ["mongodb", "python", "pymongo"],
    "date": datetime.datetime.utcnow()
}
post_id = posts.insert_one(post).inserted_id
post_id

ObjectId('5c60410ea1107a07f357959b')

In [6]:
db.list_collection_names()

['posts']

In [7]:
pprint(posts.find_one())

{'_id': ObjectId('5c60410ea1107a07f357959b'),
 'author': 'Mike',
 'date': datetime.datetime(2019, 2, 10, 15, 19, 42, 916000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


You can launch a terminal aside, connect to your server with a mongo client and check that the value is present :

```bash
vagrant@nosql:~$ mongo
> show databases;
admin          0.000GB
config         0.000GB
local          0.000GB
test_database  0.000GB
> use test_database;
switched to db test_database
> db.posts.find()
{ 
    "_id" : ObjectId("..."), 
    "author" : "Mike", 
    "text" : "My first blog post!", 
    "tags" : [ "mongodb", "python", "pymongo"], 
    "date" : ISODate("2019-02-10T11:33:47.883Z") 
}
```

## I. Quick start

### First steps

**Q** : Create a document `{msg: 'hello'}` in the `test` collection with `insert_one()`. Fetch it back to display it. What is the `_id` for ?

NB : if the collection doesn't exist yet, MongoDB automatically creates it.

In [8]:
db.test.insert_one({'msg': 'hello'})

In [9]:
db.test.find_one()

{'_id': ObjectId('5c60410fa1107a07f357959c'), 'msg': 'hello'}

**Q**: Display the number of documents inside the `test` collection

In [10]:
db.test.count_documents({})

1

### Interacting with a database

We have 2 `.json` files we want to interact with inside the `data` folder. Let's first dump them into a `MovieLens` database, inside `users` and `movies` collections.

For this section, you will need to read a bit on [query operators](https://docs.mongodb.com/manual/reference/operator/query/#query-selectors). Most methods on collections you will use have `filter` as a first parameter, on which you must pass a dictionary of query parameters.

**Q** : In the `MovieLens` database, load `data/movielens_movies.json` into `movies` and `data/movielens_users.json` into `users`. 

Use the dedicated shell command for this : `mongoimport --db <some_db> --collection <some_collection> --file <some_file>` 

In [11]:
!mongoimport --db MovieLens --collection users --file ../data/movielens_users.json

2019-02-10T15:19:43.198+0000	connected to: localhost
2019-02-10T15:19:46.196+0000	[#####################...] MovieLens.users	52.4MB/58.6MB (89.4%)
2019-02-10T15:19:46.581+0000	[########################] MovieLens.users	58.6MB/58.6MB (100.0%)
2019-02-10T15:19:46.582+0000	imported 6040 documents


In [12]:
!mongoimport --db MovieLens --collection movies --file ../data/movielens_movies.json

2019-02-10T15:19:46.777+0000	connected to: localhost
2019-02-10T15:19:46.858+0000	imported 3883 documents


**Q** : how many users are in the `MovieLens` database ?

In [13]:
movielens_db = client.MovieLens
users_col = movielens_db.users
movies_col = movielens_db.movies

users_col.count_documents({})

6040

**Q** : Display all comedies (the `genres` property equals `Comedy`). 

NB : You will need to find how to go through a `command_cursor`, then use the `pprint` function for a better display of those documents.

In [14]:
for doc in movies_col.find({'genres': 'Comedy'}, limit=10):
    pprint(doc)

{'_id': 5, 'genres': 'Comedy', 'title': 'Father of the Bride Part II (1995)'}
{'_id': 19,
 'genres': 'Comedy',
 'title': 'Ace Ventura: When Nature Calls (1995)'}
{'_id': 38, 'genres': 'Comedy', 'title': 'It Takes Two (1995)'}
{'_id': 52, 'genres': 'Comedy', 'title': 'Mighty Aphrodite (1995)'}
{'_id': 63,
 'genres': 'Comedy',
 'title': "Don't Be a Menace to South Central While Drinking Your Juice in the "
          'Hood (1996)'}
{'_id': 65, 'genres': 'Comedy', 'title': 'Bio-Dome (1996)'}
{'_id': 69, 'genres': 'Comedy', 'title': 'Friday (1995)'}
{'_id': 88, 'genres': 'Comedy', 'title': 'Black Sheep (1996)'}
{'_id': 96, 'genres': 'Comedy', 'title': 'In the Bleak Midwinter (1995)'}
{'_id': 101, 'genres': 'Comedy', 'title': 'Bottle Rocket (1996)'}


**Q** : Fetch and display the `name` and `occupation` for Clifford Johnathan. The second paramater for `find()` ([doc here](https://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.find)) is called the `projection` and is used to limit which data to fetch from the query.

In [15]:
users_col.find({'name': 'Clifford Johnathan'}, {'name': 1, 'occupation': 1}).next()

{'_id': 1276,
 'name': 'Clifford Johnathan',
 'occupation': 'technician/engineer'}

**Q**: How many minors (by `age`) have rated movies ?

In [16]:
users_col.count_documents({'age': {'$lt': 18}})

222

**Q**: Display science fiction movies ('Sci-Fi') and suspense movies ('Thriller'). This time you need to use a regex to parse genres and look for those values.

In [17]:
for doc in movies_col.find({'$or': [{'genres':{'$regex':'Thriller'}}, {'genres':{'$regex':'Sci-Fi'}}]}, limit=10):
    pprint(doc)

{'_id': 6, 'genres': 'Action|Crime|Thriller', 'title': 'Heat (1995)'}
{'_id': 10, 'genres': 'Action|Adventure|Thriller', 'title': 'GoldenEye (1995)'}
{'_id': 16, 'genres': 'Drama|Thriller', 'title': 'Casino (1995)'}
{'_id': 18, 'genres': 'Thriller', 'title': 'Four Rooms (1995)'}
{'_id': 22, 'genres': 'Crime|Drama|Thriller', 'title': 'Copycat (1995)'}
{'_id': 23, 'genres': 'Thriller', 'title': 'Assassins (1995)'}
{'_id': 24, 'genres': 'Drama|Sci-Fi', 'title': 'Powder (1995)'}
{'_id': 29,
 'genres': 'Adventure|Sci-Fi',
 'title': 'City of Lost Children, The (1995)'}
{'_id': 32, 'genres': 'Drama|Sci-Fi', 'title': 'Twelve Monkeys (1995)'}
{'_id': 47, 'genres': 'Crime|Thriller', 'title': 'Seven (Se7en) (1995)'}


**Q**: If we want more advanced textual search, we need a particular index. Use the `create_index()` method to index as [TEXT](https://docs.mongodb.com/manual/core/index-text/) the `genres` field of the `movies` collection.

In [18]:
movies_col.create_index([('genres', pymongo.TEXT)])

'genres_text'

**Q**: Restart the search for science fiction and thriller movies with the operator `$text`

In [19]:
for doc in movies_col.find({'$text':{'$search':'Sci-Fi Thriller'}}, limit=10):
    pprint(doc)

{'_id': 3934, 'genres': 'Sci-Fi', 'title': 'Kronos (1957)'}
{'_id': 3878, 'genres': 'Sci-Fi', 'title': 'X: The Unknown (1956)'}
{'_id': 3780, 'genres': 'Sci-Fi', 'title': 'Rocketship X-M (1950)'}
{'_id': 3779, 'genres': 'Sci-Fi', 'title': 'Project Moon Base (1953)'}
{'_id': 3687, 'genres': 'Sci-Fi', 'title': 'Light Years (1988)'}
{'_id': 3658, 'genres': 'Sci-Fi', 'title': 'Quatermass and the Pit (1967)'}
{'_id': 3486, 'genres': 'Sci-Fi', 'title': 'Devil Girl From Mars (1954)'}
{'_id': 3375, 'genres': 'Sci-Fi', 'title': 'Destination Moon (1950)'}
{'_id': 3354, 'genres': 'Sci-Fi', 'title': 'Mission to Mars (2000)'}
{'_id': 2665,
 'genres': 'Sci-Fi',
 'title': 'Earth Vs. the Flying Saucers (1956)'}


**Q**: Display the first 30 movies (`limit`) in alphabetical order (`sort`) by title

In [20]:
for doc in movies_col.find(filter={}, sort=[('title', 1)], limit=30):
    pprint(doc)

{'_id': 2031, 'genres': "Children's|Comedy", 'title': '$1,000,000 Duck (1971)'}
{'_id': 3112, 'genres': 'Drama', 'title': "'Night Mother (1986)"}
{'_id': 779, 'genres': 'Drama|Romance', 'title': "'Til There Was You (1997)"}
{'_id': 2072, 'genres': 'Comedy', 'title': "'burbs, The (1989)"}
{'_id': 3420,
 'genres': 'Drama|Thriller',
 'title': '...And Justice for All (1979)'}
{'_id': 889, 'genres': 'Romance', 'title': '1-900 (1994)'}
{'_id': 2572,
 'genres': 'Comedy|Romance',
 'title': '10 Things I Hate About You (1999)'}
{'_id': 2085,
 'genres': "Animation|Children's",
 'title': '101 Dalmatians (1961)'}
{'_id': 1367, 'genres': "Children's|Comedy", 'title': '101 Dalmatians (1996)'}
{'_id': 1203, 'genres': 'Drama', 'title': '12 Angry Men (1957)'}
{'_id': 2826,
 'genres': 'Action|Horror|Thriller',
 'title': '13th Warrior, The (1999)'}
{'_id': 1609, 'genres': 'Drama', 'title': '187 (1997)'}
{'_id': 999, 'genres': 'Crime', 'title': '2 Days in the Valley (1996)'}
{'_id': 2492, 'genres': 'Comedy

**Q**: How many users have seen the movie "Star Wars: Episode V - The Empire Strikes Back (1980)" (`_id 1196`) ? The `movies` argument is an array so we should try the [elemMatch](https://docs.mongodb.com/manual/reference/operator/projection/elemMatch/) operator here.

In [21]:
users_col.count_documents({'movies': {'$elemMatch': {'movieid': 1196}}})

2990

**Q**: And how many gave it a rating of 1 or 2 ?

In [22]:
users_col.count_documents({'movies': {'$elemMatch': {'movieid': 1196, 'rating': {'$lte': 2}}}})

105

### Updating data

**Q**: Insert a new user with the properties `name`, `gender` ('M' or'F'), `occupation` and `age`, using the `insert_one()` command. Display it with `find_one()`.

In [23]:
users_col.insert_one({'name':'john woo', 'age': 70, 'gender': 'M', 'occupation': 'movie maker'})

In [24]:
users_col.find_one({'name':'john woo'})

{'_id': ObjectId('5c604113a1107a07f357959d'),
 'name': 'john woo',
 'age': 70,
 'gender': 'M',
 'occupation': 'movie maker'}

**Q**: Add an appreciation on a viewed movie with `update_one()`, add the movies property containing a table with a document (`movieid`, `rating`, `timestamp` with the value `datetime.datetime.utcnow()`).

You will need to read the documentation on [update operators](https://docs.mongodb.org/manual/reference/operator/update/).

In [25]:
users_col.update_one({'name': 'john woo'}, {'$set':{'movies':[{'movieid':1196, 'rating': 5, 'timestamp':datetime.datetime.utcnow()}]}})

In [26]:
users_col.find_one({'name':'john woo'})

{'_id': ObjectId('5c604113a1107a07f357959d'),
 'name': 'john woo',
 'age': 70,
 'gender': 'M',
 'occupation': 'movie maker',
 'movies': [{'movieid': 1196,
   'rating': 5,
   'timestamp': datetime.datetime(2019, 2, 10, 15, 19, 47, 701000)}]}

**Q**: Find the number of users who have declared a `programmer` occupation. Modify them so that they are `developer`. Verify your update.

In [27]:
users_col.update_many({'occupation': 'programmer'}, {'$set': {'occupation': 'developer'}})

In [28]:
users_col.count_documents({'occupation': 'developer'})

388

## II. Modelling a blog

We will now model a blog using Mongo. 

First, switch to a new `Blog` database. Each blog post will have the following arguments:

* The author (author field, string type)
* The date (date field, string type in YYYY-MM-DD format)
* The content (field content)
* Tags (field tags, a string array)
* A list of comments (field comments) containing:
 * The author (author field, string type)
 * The date (date field, string type in YYYY-MM-DD format)
 * The content (field content)


**Q**: Create a first post by `rick`, on January 15th, with the tags `mongodb` and `nosql`.

**Q**: Create a second post by `kate`, on January 21, with the tag `nosql` and a comment from `rick` on the same day.

**Q**: Display the author of the last post with the tag `nosql`

**Q**: Add a comment by `jack` on January 25, to `kate`'s post

**Q**: Display all comments by `kate`

## Postquisites

In [29]:
!mongo test_database --eval 'db.dropDatabase()'

MongoDB shell version v4.0.6
connecting to: mongodb://127.0.0.1:27017/test_database?gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("f5a7506b-1b49-43ef-8a31-e1be465931bc") }
MongoDB server version: 4.0.6
{ "dropped" : "test_database", "ok" : 1 }


In [30]:
!mongo MovieLens --eval 'db.dropDatabase()'

MongoDB shell version v4.0.6
connecting to: mongodb://127.0.0.1:27017/MovieLens?gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("d79acb93-617f-40c7-b82b-6b5cbcb3568b") }
MongoDB server version: 4.0.6
{ "dropped" : "MovieLens", "ok" : 1 }


In [31]:
!mongo Blog --eval 'db.dropDatabase()'

MongoDB shell version v4.0.6
connecting to: mongodb://127.0.0.1:27017/Blog?gssapiServiceName=mongodb
Implicit session: session { "id" : UUID("cee4f347-6f42-4f77-95e0-e5fa85c87957") }
MongoDB server version: 4.0.6
{ "ok" : 1 }
